In [1]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from random import randrange
import time
#from nltk.stem.porter import PorterStemmer
import spacy
from nltk.stem import WordNetLemmatizer
#nltk.download('all')
nlp = spacy.load('en_core_web_lg',  disable=["parser", "ner"])
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
from sklearn.metrics.pairwise import cosine_similarity
from nltk import pos_tag
wnl = WordNetLemmatizer()
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 12.324640989303589 secs.


In [2]:
import requests

url = "https://api-football-beta.p.rapidapi.com/players/topscorers"

querystring = {"season":"2022","league":"39"}

headers = {
    "X-RapidAPI-Key": "62a02ca5ccmsha68a5acf6c698d9p1ffc12jsnbe2b9a765139",
    "X-RapidAPI-Host": "api-football-beta.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#print (type(response.json()))
#print(response.json())
df = response.json()

In [19]:
playerid = 0
goals = df['response'][playerid]['statistics'][0]['goals']['total']
shot = df['response'][playerid]['statistics'][0]['shots']['total']
passing = df['response'][playerid]['statistics'][0]['passes']['total']
playercode = {'Earling Haaland':0, ' Harry Kane':1, 'Aleksandar Mitrovic':2, 'Ivan Toney':3, 'Leandro Trossard':4, ' M. Miguel Almiron':5, 'Phil Foden':6, 'Roberto Firmino':7}

In [4]:
def askplayer(name):
    askq = input("If you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? ")
    if askq =='yes' or askq == 'Yes':
        print("Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them")
        print("Earling Haaland, Harry Kane, Aleksandar Mitrovic, Ivan Toney, Leandro Trossard, M. Miguel Almiron, Phil Foden, Roberto Firmino")  
       #playerchoose = input("So what's your choice ")
        while True:
            playerchoose = input("So what's your choice ")
            if playerchoose in playercode.keys():
                playerid = (playercode[playerchoose])
                #print(playerid)
                print("hmm...")
                time.sleep(1)
                print("Ah yes... " + playerchoose + " I know all about him. Did you know in this season he has scored a total of " 
                      + str(goals) + " goals and he's attempted a grande total of " + str(shot) +" shots")
                #return False
                break
            else:
                print("That's not one of the names I gave you... Try again.")

In [5]:
 def word_tokenization(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text_tokenized = word_tokenize(text)
    
    for word in text_tokenized:
        if word in stopwords.words('english'):
            text_tokenized.remove(word)
        
    return(text_tokenized)


In [6]:
def taggandlem(text):
    global lemmatized_list 
    global lmt_string
    entity_list = []
    for word in text:
        if 'NN' in word[1] or 'JJ' in word[1]  or 'VB' in word[1]:
            entity_list.append(word[0])
    
    lemmatized_list = []
    for lit in entity_list:
        lmtz = wnl.lemmatize(lit)
        lemmatized_list.append(lmtz)
        lmt_string = ' '.join(lemmatized_list)

In [22]:
def lem_response_joined(text):
    global lemlist
    number = 0
    lemlist = []
    n=0
    for word in text:
        number = number+1
     
    while n<number:
        gg =' '.join(text[n])
        lemlist.append(gg)
        n=n+1
    
    #print(lemlist)
    

In [8]:
question = [" ",
            "hello, hey, hi, how are you",
            "what is your favourite team",
            "what is your least favourite or most hated team",
            "how long have you been watching football"
                    
                   ]

In [9]:
response = ["Ask another question",
            'Hey, my name is Garry I will be answering your football questions, feel free to ask anything',
                  'My favourite team is Manchester United',
                  'I really hate Manchester City',
                 "I've been watching football since I was two, for as long as I remember I've loved football",    
                 ]

In [10]:
question_list = []
for word in question:
    question_list.append(lemmatize_sentence(word,))
    
lem_response_joined(question_list)
question_lem_list = lemlist

In [11]:
chat_frame = pd.DataFrame()
chat_frame['question'] = question_lem_list
chat_frame['response'] = response

In [12]:
#chat_frame

In [13]:
def response_selection(ui):
    sentence_input = [(ui)]
    similarity_index_list = cosine_similarity(v.fit_transform(chat_frame["question"]), v.transform(sentence_input)).flatten()
    output = chat_frame.loc[similarity_index_list.argmax(), "response"]
    print(output)

In [14]:
def speech(text):
    tag = True
    #exit_command = ['bye','exit,','goodbye']
    while tag == True:
        user_input = input()
        user_input = user_input.lower()
        if user_input == "bye" or  user_input == "exit" or  user_input == "goodbye":
            print("goodbye")
            tag = False
        
        elif "top scorer" in user_input:
            askplayer("name")
            
        else:
            tokenized_respone = word_tokenization(user_input)
            tagged_response = pos_tag(tokenized_respone)
            taggandlem(tagged_response)
            user_input = lmt_string
            #print(user_input)
            response_selection(user_input)

In [16]:
speech("hello")

hey
Hey, my name is Garry I will be answering your football questions, feel free to ask anything
top scorer
If you want I can tell you about some of the top scorers in the premiere league. So what do you say, Yes or No? yes
Here are the names of some of the top scorers I know, Choose any of those names and I'll tell you all about them
Earling Haaland, Harry Kane, Aleksandar Mitrovic, Ivan Toney, Leandro Trossard, M. Miguel Almiron, Phil Foden, Roberto Firmino
So what's your choice Harry Kane
That's not one of the names I gave you... Try again.
So what's your choice Harry Kane
That's not one of the names I gave you... Try again.
So what's your choice Aleksandar Mitrovic
hmm...
Ah yes... Aleksandar Mitrovic I know all about him. Did you know in this season he has scored a total of 18 goals and he's attempted a grande total of 43 shots
yes
Ask another question

Ask another question
bye
goodbye
